<a href="https://colab.research.google.com/github/muhozag/sparklearning/blob/clustering/ClusteringConsult.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Hacker Identification Clustering Example**

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('hacker_id').getOrCreate()

In [3]:
dataset = spark.read.csv('hack_data.csv', header=True, inferSchema=True)

In [4]:
dataset.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [5]:
dataset.head()

Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)

In [6]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

In [7]:
dataset.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [8]:
feature_cols = ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed']

In [9]:
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')

In [10]:
final_data = assembler.transform(dataset)

In [11]:
final_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)



In [12]:
from pyspark.ml.feature import StandardScaler

In [13]:
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures')

In [14]:
scaler_model = scaler.fit(final_data)

In [15]:
cl_final_data = scaler_model.transform(final_data)

In [16]:
cl_final_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- scaledFeatures: vector (nullable = true)



# Was it 2 hackers or 3

In [18]:
kmeans2 = KMeans(featuresCol='scaledFeatures', k=2)
kmeans3 = KMeans(featuresCol='scaledFeatures', k=3)

In [19]:
model_k2 = kmeans2.fit(cl_final_data)
model_k3 = kmeans3.fit(cl_final_data)

In [20]:
model_k2.transform(cl_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



In [21]:
model_k3.transform(cl_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   83|
|         2|  167|
|         0|   84|
+----------+-----+



**It was 2 hackers**